In [85]:
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [32]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [47]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [49]:
wiki_tool.name

'wikipedia'

In [35]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import  FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

In [43]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorDB = FAISS.from_documents(documents, embeddings) 
retriever = vectorDB.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000019FEA66DBE0>, search_kwargs={})

In [ ]:
from langchain.tools.retriever import create_retriever_tool 

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool",
)

retriever_tool.name

'langsmith_search'

In [74]:

loader = WebBaseLoader("https://www.csc.jfn.ac.lk/students-csc-15/")
docs=loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorDB = FAISS.from_documents(documents, embeddings) 
retriever = vectorDB.as_retriever()
retriever


from langchain.tools.retriever import create_retriever_tool 

csc_tool = create_retriever_tool(
    retriever,
    name="csc_students",
    description=(
        "Search for information about University of Jaffna Computer Science students. "
        "For any questions about University of Jaffna Computer Science students, use this tool."
    ),
)

csc_tool.name

'csc_students'

In [46]:
# Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)

arxiv.name

'arxiv'

In [75]:
tools = [wiki_tool,arxiv,retriever_tool,csc_tool]

In [76]:
from langchain_groq import ChatGroq


llm = ChatGroq(model_name="llama3-8b-8192", temperature=0, groq_api_key=os.getenv("GROQ_API_KEY"))

In [86]:
from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

In [87]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\Users\Nipun Weerasinghe\Desktop\doing_projects\venv\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [88]:
# Agents

from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm,tools,prompt)

In [89]:
## Agent Executer

from langchain.agents import AgentExecutor

agent_executer = AgentExecutor(agent=agent,tools=tools,verbose=True)

In [90]:
agent_executer.invoke({"input":"Tell me about few names of university of jaffna computer science students Academic Year: 2019/2020"})



> Entering new AgentExecutor chain...

Invoking: `csc_students` with `{'query': 'University of Jaffna Computer Science students names Academic Year: 2019/2020'}`


Students-CSC-15 – Department of Computer Science











































Skip to content






UoJFoSLMSCS-SPMS






















Search...

Submit search











HomeAbout

About Us New Building

Overview Donations Photo Gallery 

DegreesSyllabus

Direct-Intake Physical Science Applied Science
Staff

Present Staff Former Heads Former Staff Dept. meetings
Students

Direct Intake Physical Science MSc MPhil PhD
Research

Publications Undergraduate Research Projects Postgraduate Research Projects Grants Fellowships Dataset Google’s exploreCSR
Awards

Achievements Dean’s List Prizes

Degree-based Level-based Staff-Prizes 

Activities

Conferences/Workshops CS-Career Fair Industrial Visit Participations Outreach Activities TechTalk/Webinar Summer Schools

GeSSoMI – 2024 IEEE SSCI – 2023 

GalleryAlumni

{'input': 'Tell me about few names of university of jaffna computer science students Academic Year: 2019/2020',
 'output': 'I am sorry, I cannot fulfill this request. I do not have access to a list of student names for the academic year 2019/2020. The information I found only confirms the commencement date of the batch 15 as September 2021.'}

In [91]:
agent_executer.invoke({"input":"Tell me about Machine Learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine Learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genMachine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data.

> Finished chain.


{'input': 'Tell me about Machine Learning',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data.'}